# Imports

In [1]:
import nfl_data_py as nfl
import pandas as pd

# Loading PBP df

In [2]:
years = range(1999, 2023 + 1)

In [3]:
df = nfl.import_pbp_data(years)

1999 done.
2000 done.
2001 done.
2002 done.
2003 done.
2004 done.
2005 done.
2006 done.
2007 done.
2008 done.
2009 done.
2010 done.
2011 done.
2012 done.
2013 done.
2014 done.
2015 done.
2016 done.
2017 done.
2018 done.
2019 done.
2020 done.
2021 done.
2022 done.
2023 done.
Downcasting floats.


In [4]:
df.head()

,play_id,game_id,old_game_id,home_team,away_team,season_type,week,posteam,posteam_type,defteam,...,offense_players,defense_players,n_offense,n_defense,ngs_air_yards,time_to_throw,was_pressure,route,defense_man_zone_type,defense_coverage_type
0,35.0,1999_01_ARI_PHI,1999091200,PHI,ARI,REG,1,PHI,home,ARI,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,60.0,1999_01_ARI_PHI,1999091200,PHI,ARI,REG,1,PHI,home,ARI,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,82.0,1999_01_ARI_PHI,1999091200,PHI,ARI,REG,1,PHI,home,ARI,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,103.0,1999_01_ARI_PHI,1999091200,PHI,ARI,REG,1,PHI,home,ARI,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,126.0,1999_01_ARI_PHI,1999091200,PHI,ARI,REG,1,PHI,home,ARI,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Grabbing Redzone Targets

In [5]:
df_redzone = df.query("yardline_100 <= 20 and play_type == 'pass' and season_type == 'REG'")[['season', 
                                                                                               'receiver_player_name', 'receiver_id']]

In [6]:
df_redzone_targets = df_redzone \
    .groupby(by=['season', 'receiver_id']) \
    .agg(redzone_targets=('receiver_player_name', "count"))

df_redzone_targets.reset_index(inplace=True)
df_redzone_targets.rename(columns={"receiver_id": "player_id"}, inplace=True)

In [7]:
df_redzone_targets.head()

,season,player_id,redzone_targets
0,1999,00-0000003,3
1,1999,00-0000145,1
2,1999,00-0000166,7
3,1999,00-0000172,1
4,1999,00-0000210,2


# Loading Core df

In [8]:
df_core = pd.read_pickle("../../FantasyData/data-frames/df_basic_ngs_snaps_adv_1999_2023.pkl")

In [9]:
df_core.head()

,player_id,season,season_type,completions,attempts,passing_yards,passing_tds,interceptions,sacks,sack_yards,...,yds_pfr_rush,td_pfr_rush,x1d_pfr_rush,ybc_pfr_rush,ybc_att_pfr_rush,yac_pfr_rush,yac_att_pfr_rush,brk_tkl_pfr_rush,att_br_pfr_rush,loaded_pfr_rush
0,00-0000003,1999,REG,0,0,0.0,0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,00-0000003,1999,REG,0,0,0.0,0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,00-0000003,2000,REG,0,0,0.0,0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,00-0000007,1999,REG,0,0,0.0,0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,00-0000007,2000,REG,0,0,0.0,0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Merging dfs

In [10]:
df_merge = df_core.copy().merge(df_redzone_targets, on=["season", "player_id"], how="left")

In [11]:
df_merge.query("redzone_targets >= 30")[["season", "player_name", "redzone_targets", "targets"]].head()

,season,player_name,redzone_targets,targets
989,2000,Marvin Harrison,31.0,168
1662,2002,Randy Moss,36.0,184
1677,2002,Eric Moulds,31.0,180
5089,2011,Roddy White,30.0,180
5747,2009,Brandon Marshall,30.0,154


In [12]:
df_merge['redzone_targets'].fillna(0, inplace=True)

/var/folders/yx/gjm2fjs56vl5kkwrb6xf5d2h0000gn/T/ipykernel_31731/2923396243.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_merge['redzone_targets'].fillna(0, inplace=True)


In [14]:
df_merge.query("redzone_targets >= 30")[["season", "player_name", "redzone_targets", "targets"]].head()

,season,player_name,redzone_targets,targets
989,2000,Marvin Harrison,31.0,168
1662,2002,Randy Moss,36.0,184
1677,2002,Eric Moulds,31.0,180
5089,2011,Roddy White,30.0,180
5747,2009,Brandon Marshall,30.0,154


# Save New df

In [15]:
df_merge.to_pickle("../../FantasyData/data-frames/df_basic_ngs_snaps_adv_redzone-tgts_1999_2023.pkl")